C:\Users\tomsm\PycharmProjects\thesis\results\simclr_graph


In [13]:
from utils.helper_functions import load_model, prepare_data_features
from models.simclr_model import SimCLR
from models.supervised_model import SupervisedModel
from datasets.SHHS_dataset_timeonly import SHHS_dataset_1
import torch.utils.data as data
from copy import deepcopy
import torch.nn as nn
import torch

In [8]:
encoder_path = "../../trained_models/cnn_simclr_500pat.ckpt"
pretrained_model = load_model(SimCLR, encoder_path)  # Load pretrained simclr model

C:\Users\tomsm\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
C:\Users\tomsm\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'projector' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['projector'])`.
  rank_zero_warn(


In [11]:
test_ds = SHHS_dataset_1(data_path='../../../thesis01/data/',
                             first_patient=1,
                             num_patients=30)
test_dl = data.DataLoader(dataset=test_ds,
                              batch_size=64,
                              shuffle=False,
                              num_workers=12)

Couldn't find file at path:  ../../../thesis01/data/n0014_eeg.mat


In [16]:
device = 'cpu'
test_ds = prepare_data_features(model=pretrained_model,
                                             data_loader=test_dl,
                                             device=device)

test_dl = data.DataLoader(dataset=test_ds,
                                   batch_size=64,
                                   shuffle=False,
                                   num_workers=12)

  0%|          | 0/444 [00:00<?, ?it/s]

In [15]:
ckpt_path = ""
sup_model = load_model(SupervisedModel, ckpt_path)
sup_model

RuntimeError: Error(s) in loading state_dict for SupervisedModel:
	Missing key(s) in state_dict: "encoder.model.0.net.0.weight", "encoder.model.0.net.1.weight", "encoder.model.0.net.1.bias", "encoder.model.0.net.1.running_mean", "encoder.model.0.net.1.running_var", "encoder.model.1.net.0.weight", "encoder.model.1.net.1.weight", "encoder.model.1.net.1.bias", "encoder.model.1.net.1.running_mean", "encoder.model.1.net.1.running_var", "encoder.model.2.net.0.weight", "encoder.model.2.net.1.weight", "encoder.model.2.net.1.bias", "encoder.model.2.net.1.running_mean", "encoder.model.2.net.1.running_var", "encoder.model.4.weight", "encoder.model.4.bias", "classifier.weight", "classifier.bias". 

In [ ]:
network = deepcopy(sup_model.encoder)
network.g = nn.Identity()  # Removing projection head g(.)
network.eval()
network.to(device)

# Encode all images
feats, labels = [], []
for batch_inputs, batch_labels in test_dl:
    batch_inputs = batch_inputs.to(device)
    batch_feats = network(batch_inputs.squeeze(dim=1))
    feats.append(batch_feats.detach().cpu())
    labels.append(batch_labels)

feats = torch.cat(feats, dim=0)
labels = torch.cat(labels, dim=0)

test_features_ds = data.TensorDataset(feats, labels)
test_features_dl = data.DataLoader(dataset=test_features_ds,
                                       batch_size=64,
                                       shuffle=False,
                                       num_workers=12)